In [ ]:
pip install requests

In [ ]:
import numpy as np
import pandas as pd
import ssl
import gme
import requests
import io
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [ ]:
# load USITC trade data directly from online
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.5060.53 Safari/537.36'}
temp = requests.get('https://www.usitc.gov/data/gravity/example_trade_and_grav_data_small.csv', headers=headers).content
gravity_data = pd.read_csv(io.StringIO(temp.decode('utf-8')))

In [ ]:
# check some sample data
gravity_data.loc[(gravity_data['importer']=='CAN') & (gravity_data['exporter']=='USA') & (gravity_data['year']==1999)]

In [ ]:
# check some sample data
gravity_data.loc[(gravity_data['importer']=='USA') & (gravity_data['exporter']=='SGP') & (gravity_data['year']==1999)]

In [ ]:
# make data compatable with gme modulejk
gme_data = gme.EstimationData(data_frame=gravity_data,
                          imp_var_name='importer',
                          exp_var_name='exporter',
                          trade_var_name='trade_value',
                          year_var_name='year',
                          notes='Downloaded from https://www.usitc.gov/data/gravity/example_trade_and_grav_data_small.csv')

gme_data

In [ ]:
print(gme_data.number_of_importers)
print(gme_data.columns)
print(gme_data.year_list())
country_list = gme_data.countries_each_year()
print(country_list[1989])

print(gme_data.dtypes())
print(gme_data.info())
print(gme_data.describe())

How to interpret results from the model output
https://www.statsmodels.org/devel/generated/statsmodels.genmod.generalized_linear_model.GLMResults.html

In [ ]:
model_baseline = gme.EstimationModel(estimation_data= gme_data,
                                 lhs_var = 'trade_value',
                                 rhs_var = ['log_distance','agree_pta',
                                            'common_language','contiguity'])
model_baseline.estimate()
print(model_baseline.results_dict['all'].summary())

In [ ]:
estimate = model_baseline.estimate()
result = estimate['all']
# extract fitted coefficient from the model
coefficients = result.params
print(coefficients)

In [ ]:
# extract pValue and pearson_chi2 values from the result
print(f'p-value = {result.pvalues}')
print(f'pearson_chi2 = {result.pearson_chi2}')

In [ ]:
# looking at grouped year variation
for year_combo in ([2000, 2001, 2002], [2003, 2004, 2005], [2006, 2007, 2008], [2009, 2010, 2011], [2012, 2013, 2014], [2015, 20016, 20017]):
    model = gme.EstimationModel(estimation_data= gme_data,
                                    lhs_var = 'trade_value',
                                    rhs_var = ['log_distance','agree_pta',
                                                'common_language','contiguity'],
                                    keep_years = year_combo)
    print(year_combo)
    # model.estimate()
    # print(model.results_dict['all'].summary())
    estimate = model.estimate()
    result = estimate['all']
    coefficients = result.params
    print(f'coeff = {coefficients[:3]}')
    # print(f'pvalues = {result.pvalues}')

In [ ]:
# additional examples
fixed_effects_model  = gme.EstimationModel(estimation_data= gme_data,
                                       lhs_var = 'trade_value',
                                       rhs_var = ['log_distance','agree_pta',
                                            'common_language','contiguity'],
                                       fixed_effects=[['importer','year'],
                                                ['exporter','year']])
fixed_effects_model.estimate()
#print(fix_effects_model.results_dict['all'].summary())

In [ ]:
# additional examples
data_subset_model = gme.EstimationModel(estimation_data= gme_data,
                                    lhs_var = 'trade_value',
                                    rhs_var = ['log_distance','agree_pta',
                                            'common_language','contiguity'],
                                    drop_imp_exp=['USA'],
                                    keep_years=[2015, 2014, 2013])

In [ ]:
# additional examples
fixed_effects_model_2  = gme.EstimationModel(estimation_data= gme_data,
                                         lhs_var = 'trade_value',
                                         rhs_var = ['log_distance','agree_pta',
                                            'common_language','contiguity'],
                                         fixed_effects=[['importer','year'],
                                                ['exporter','year']],
                                         keep_years=[2013,2014,2015])

In [ ]:
# explanatin on fixed-effect estimation
# https://www.unescap.org/sites/default/d8files/5%20-%203.%20Estimating%20the%20Gravity%20Model_0.pdf
estimates = fixed_effects_model_2.estimate()
results = estimates['all']
results.summary()

In [ ]:
coefficients = results.params
coefficients

In [ ]:
results.fittedvalues

In [ ]:
fixed_effects_model_2.ppml_diagnostics
# fixed_effects_model_2.ppml_diagnostics['Perfectly Collinear Variables']

In [ ]:
combined_results = fixed_effects_model_2.combine_sector_results()
combined_results.head()

In [ ]:
# more examples
fixed_effects_model_2.format_regression_table(format='csv',
                                            se_below = True,
                                            omit_fe_prefix = ['importer_year',
                                                                   'exporter_year'])

## with other distances

### political

In [ ]:
agreement_score = pd.read_csv('AgreementScoresAll_Jun2022.csv\AgreementScoresAll_Jun2022.csv')
agreement_score.head()
cow2iso = pd.read_csv('cow2iso.csv')
cow2iso.head()
agreement_score_iso3 = agreement_score.merge(cow2iso, left_on='ccode1', right_on='cow_id').merge(cow2iso, left_on='ccode2', right_on='cow_id')[['iso3_x', 'iso3_y', 'year', 'IdealPointDistance']]
agreement_score_iso3.head()

In [ ]:
gravity_data = gravity_data.merge(agreement_score_iso3, left_on=['importer','exporter', 'year'],
             right_on=['iso3_x', 'iso3_y', 'year'])
gravity_data.head()

In [ ]:
gravity_data['political_distance'] = np.log(gravity_data['IdealPointDistance'])

In [128]:
gravity_data.head()

,importer,exporter,year,trade_value,agree_pta,common_language,contiguity,log_distance,iso3_x,iso3_y,IdealPointDistance,political_distance,ctr_x,country_x,pdi_x,idv_x,mas_x,uai_x,ltowvs_x,ivr_x,cow_id_x,cow3_x,iso_id_x,iso2_x,iso3_x,valid_from_x,valid_until_x,cname_x,cname_full_x,comments_x,statenme_x,ctr_y,country_y,pdi_y,idv_y,mas_y,uai_y,ltowvs_y,ivr_y,cow_id_y,cow3_y,iso_id_y,iso2_y,iso3_y,valid_from_y,valid_until_y,cname_y,cname_full_y,comments_y,statenme_y,ctr_x,country_x,pdi_x,idv_x,mas_x,uai_x,ltowvs_x,ivr_x,cow_id_x,cow3_x,iso_id_x,iso2_x,iso3_x,valid_from_x,valid_until_x,cname_x,cname_full_x,comments_x,statenme_x,ctr_y,country_y,pdi_y,idv_y,mas_y,uai_y,ltowvs_y,ivr_y,cow_id_y,cow3_y,iso_id_y,iso2_y,iso3_y,valid_from_y,valid_until_y,cname_y,cname_full_y,comments_y,statenme_y
0,AUS,BRA,1989,303546944.0,0.0,1.0,0.0,9.553332,AUS,BRA,1.299100,0.261671,AUL,Australia,38.0,90.0,61.0,51.0,21.0,71.0,900.0,AUL,36.0,AU,AUS,1962.0,NaN,Australia,Australia,NaN,Australia,BRA,Brazil,69.0,38.0,49.0,76.0,44.0,59.0,140.0,BRA,76.0,BR,BRA,1962.0,NaN,Brazil,Brazil,NaN,Brazil,AUL,Australia,38.0,90.0,61.0,51.0,21.0,71.0,900.0,AUL,36.0,AU,AUS,1962.0,NaN,Australia,Australia,NaN,Australia,BRA,Brazil,69.0,38.0,49.0,76.0,44.0,59.0,140.0,BRA,76.0,BR,BRA,1962.0,NaN,Brazil,Brazil,NaN,Brazil
1,AUS,BRA,1990,228831920.0,0.0,1.0,0.0,9.553332,AUS,BRA,1.519327,0.418267,AUL,Australia,38.0,90.0,61.0,51.0,21.0,71.0,900.0,AUL,36.0,AU,AUS,1962.0,NaN,Australia,Australia,NaN,Australia,BRA,Brazil,69.0,38.0,49.0,76.0,44.0,59.0,140.0,BRA,76.0,BR,BRA,1962.0,NaN,Brazil,Brazil,NaN,Brazil,AUL,Australia,38.0,90.0,61.0,51.0,21.0,71.0,900.0,AUL,36.0,AU,AUS,1962.0,NaN,Australia,Australia,NaN,Australia,BRA,Brazil,69.0,38.0,49.0,76.0,44.0,59.0,140.0,BRA,76.0,BR,BRA,1962.0,NaN,Brazil,Brazil,NaN,Brazil
2,AUS,BRA,1991,245040080.0,0.0,1.0,0.0,9.553332,AUS,BRA,1.631600,0.489561,AUL,Australia,38.0,90.0,61.0,51.0,21.0,71.0,900.0,AUL,36.0,AU,AUS,1962.0,NaN,Australia,Australia,NaN,Australia,BRA,Brazil,69.0,38.0,49.0,76.0,44.0,59.0,140.0,BRA,76.0,BR,BRA,1962.0,NaN,Brazil,Brazil,NaN,Brazil,AUL,Australia,38.0,90.0,61.0,51.0,21.0,71.0,900.0,AUL,36.0,AU,AUS,1962.0,NaN,Australia,Australia,NaN,Australia,BRA,Brazil,69.0,38.0,49.0,76.0,44.0,59.0,140.0,BRA,76.0,BR,BRA,1962.0,NaN,Brazil,Brazil,NaN,Brazil
3,AUS,BRA,1992,224085104.0,0.0,1.0,0.0,9.553332,AUS,BRA,1.563945,0.447212,AUL,Australia,38.0,90.0,61.0,51.0,21.0,71.0,900.0,AUL,36.0,AU,AUS,1962.0,NaN,Australia,Australia,NaN,Australia,BRA,Brazil,69.0,38.0,49.0,76.0,44.0,59.0,140.0,BRA,76.0,BR,BRA,1962.0,NaN,Brazil,Brazil,NaN,Brazil,AUL,Australia,38.0,90.0,61.0,51.0,21.0,71.0,900.0,AUL,36.0,AU,AUS,1962.0,NaN,Australia,Australia,NaN,Australia,BRA,Brazil,69.0,38.0,49.0,76.0,44.0,59.0,140.0,BRA,76.0,BR,BRA,1962.0,NaN,Brazil,Brazil,NaN,Brazil
4,AUS,BRA,1993,234342624.0,0.0,1.0,0.0,9.553332,AUS,BRA,1.048615,0.047470,AUL,Australia,38.0,90.0,61.0,51.0,21.0,71.0,900.0,AUL,36.0,AU,AUS,1962.0,NaN,Australia,Australia,NaN,Australia,BRA,Brazil,69.0,38.0,49.0,76.0,44.0,59.0,140.0,BRA,76.0,BR,BRA,1962.0,NaN,Brazil,Brazil,NaN,Brazil,AUL,Australia,38.0,90.0,61.0,51.0,21.0,71.0,900.0,AUL,36.0,AU,AUS,1962.0,NaN,Australia,Australia,NaN,Australia,BRA,Brazil,69.0,38.0,49.0,76.0,44.0,59.0,140.0,BRA,76.0,BR,BRA,1962.0,NaN,Brazil,Brazil,NaN,Brazil


In [134]:
gravity_data = gravity_data.drop(columns=['iso3_x','iso3_y'])

KeyError: "['iso3_x' 'iso3_y'] not found in axis"

In [135]:
gravity_data.head()

,importer,exporter,year,trade_value,agree_pta,common_language,contiguity,log_distance,IdealPointDistance,political_distance,ctr_x,country_x,pdi_x,idv_x,mas_x,uai_x,ltowvs_x,ivr_x,cow_id_x,cow3_x,iso_id_x,iso2_x,valid_from_x,valid_until_x,cname_x,cname_full_x,comments_x,statenme_x,ctr_y,country_y,pdi_y,idv_y,mas_y,uai_y,ltowvs_y,ivr_y,cow_id_y,cow3_y,iso_id_y,iso2_y,valid_from_y,valid_until_y,cname_y,cname_full_y,comments_y,statenme_y,ctr_x,country_x,pdi_x,idv_x,mas_x,uai_x,ltowvs_x,ivr_x,cow_id_x,cow3_x,iso_id_x,iso2_x,valid_from_x,valid_until_x,cname_x,cname_full_x,comments_x,statenme_x,ctr_y,country_y,pdi_y,idv_y,mas_y,uai_y,ltowvs_y,ivr_y,cow_id_y,cow3_y,iso_id_y,iso2_y,valid_from_y,valid_until_y,cname_y,cname_full_y,comments_y,statenme_y
0,AUS,BRA,1989,303546944.0,0.0,1.0,0.0,9.553332,1.299100,0.261671,AUL,Australia,38.0,90.0,61.0,51.0,21.0,71.0,900.0,AUL,36.0,AU,1962.0,NaN,Australia,Australia,NaN,Australia,BRA,Brazil,69.0,38.0,49.0,76.0,44.0,59.0,140.0,BRA,76.0,BR,1962.0,NaN,Brazil,Brazil,NaN,Brazil,AUL,Australia,38.0,90.0,61.0,51.0,21.0,71.0,900.0,AUL,36.0,AU,1962.0,NaN,Australia,Australia,NaN,Australia,BRA,Brazil,69.0,38.0,49.0,76.0,44.0,59.0,140.0,BRA,76.0,BR,1962.0,NaN,Brazil,Brazil,NaN,Brazil
1,AUS,BRA,1990,228831920.0,0.0,1.0,0.0,9.553332,1.519327,0.418267,AUL,Australia,38.0,90.0,61.0,51.0,21.0,71.0,900.0,AUL,36.0,AU,1962.0,NaN,Australia,Australia,NaN,Australia,BRA,Brazil,69.0,38.0,49.0,76.0,44.0,59.0,140.0,BRA,76.0,BR,1962.0,NaN,Brazil,Brazil,NaN,Brazil,AUL,Australia,38.0,90.0,61.0,51.0,21.0,71.0,900.0,AUL,36.0,AU,1962.0,NaN,Australia,Australia,NaN,Australia,BRA,Brazil,69.0,38.0,49.0,76.0,44.0,59.0,140.0,BRA,76.0,BR,1962.0,NaN,Brazil,Brazil,NaN,Brazil
2,AUS,BRA,1991,245040080.0,0.0,1.0,0.0,9.553332,1.631600,0.489561,AUL,Australia,38.0,90.0,61.0,51.0,21.0,71.0,900.0,AUL,36.0,AU,1962.0,NaN,Australia,Australia,NaN,Australia,BRA,Brazil,69.0,38.0,49.0,76.0,44.0,59.0,140.0,BRA,76.0,BR,1962.0,NaN,Brazil,Brazil,NaN,Brazil,AUL,Australia,38.0,90.0,61.0,51.0,21.0,71.0,900.0,AUL,36.0,AU,1962.0,NaN,Australia,Australia,NaN,Australia,BRA,Brazil,69.0,38.0,49.0,76.0,44.0,59.0,140.0,BRA,76.0,BR,1962.0,NaN,Brazil,Brazil,NaN,Brazil
3,AUS,BRA,1992,224085104.0,0.0,1.0,0.0,9.553332,1.563945,0.447212,AUL,Australia,38.0,90.0,61.0,51.0,21.0,71.0,900.0,AUL,36.0,AU,1962.0,NaN,Australia,Australia,NaN,Australia,BRA,Brazil,69.0,38.0,49.0,76.0,44.0,59.0,140.0,BRA,76.0,BR,1962.0,NaN,Brazil,Brazil,NaN,Brazil,AUL,Australia,38.0,90.0,61.0,51.0,21.0,71.0,900.0,AUL,36.0,AU,1962.0,NaN,Australia,Australia,NaN,Australia,BRA,Brazil,69.0,38.0,49.0,76.0,44.0,59.0,140.0,BRA,76.0,BR,1962.0,NaN,Brazil,Brazil,NaN,Brazil
4,AUS,BRA,1993,234342624.0,0.0,1.0,0.0,9.553332,1.048615,0.047470,AUL,Australia,38.0,90.0,61.0,51.0,21.0,71.0,900.0,AUL,36.0,AU,1962.0,NaN,Australia,Australia,NaN,Australia,BRA,Brazil,69.0,38.0,49.0,76.0,44.0,59.0,140.0,BRA,76.0,BR,1962.0,NaN,Brazil,Brazil,NaN,Brazil,AUL,Australia,38.0,90.0,61.0,51.0,21.0,71.0,900.0,AUL,36.0,AU,1962.0,NaN,Australia,Australia,NaN,Australia,BRA,Brazil,69.0,38.0,49.0,76.0,44.0,59.0,140.0,BRA,76.0,BR,1962.0,NaN,Brazil,Brazil,NaN,Brazil


In [ ]:
gme_data = gme.EstimationData(data_frame=gravity_data,
                          imp_var_name='importer',
                          exp_var_name='exporter',
                          trade_var_name='trade_value',
                          year_var_name='year',
                          notes='Downloaded from https://www.usitc.gov/data/gravity/example_trade_and_grav_data_small.csv')

In [ ]:
fixed_effects_model_3  = gme.EstimationModel(estimation_data= gme_data,
                                         lhs_var = 'trade_value',
                                         rhs_var = ['log_distance','agree_pta',
                                            'common_language','contiguity','political_distance'],
                                         fixed_effects=[['importer','year'],
                                                ['exporter','year']],
                                         keep_years=[2013,2014,2015])

In [ ]:
estimates = fixed_effects_model_3.estimate()
results = estimates['all']
results.summary()

### cultural

In [ ]:
culture_countries = pd.read_csv('Hofstede data\\6-dimensions-for-website-2015-08-16.csv', sep=';',
                               na_values='#NULL!')
culture_countries.head()

In [120]:
culture_countries = culture_countries.merge(cow2iso, left_on='country', right_on='cname')

In [123]:
culture_countries.head()

,ctr,country,pdi,idv,mas,uai,ltowvs,ivr,cow_id,cow3,iso_id,iso2,iso3,valid_from,valid_until,cname,cname_full,comments,statenme
0,ALB,Albania,NaN,NaN,NaN,NaN,61.0,15.0,339.0,ALB,8.0,AL,ALB,1962.0,NaN,Albania,Albania,NaN,Albania
1,ALG,Algeria,NaN,NaN,NaN,NaN,26.0,32.0,615.0,ALG,12.0,DZ,DZA,1962.0,NaN,Algeria,Algeria,NaN,Algeria
2,AND,Andorra,NaN,NaN,NaN,NaN,NaN,65.0,232.0,AND,20.0,AD,AND,1962.0,NaN,Andorra,Andorra,NaN,Andorra
3,ARG,Argentina,49.0,46.0,56.0,86.0,20.0,62.0,160.0,ARG,32.0,AR,ARG,1962.0,NaN,Argentina,Argentina,NaN,Argentina
4,ARM,Armenia,NaN,NaN,NaN,NaN,61.0,NaN,371.0,ARM,51.0,AM,ARM,1992.0,NaN,Armenia,Armenia,NaN,Armenia


In [136]:
gravity_data = gravity_data.merge(culture_countries, left_on='importer', right_on='iso3').merge(culture_countries, left_on='exporter', right_on='iso3')

gravity_data.head()

C:\Users\small\AppData\Local\Temp\ipykernel_12812\3291687600.py:1: FutureWarning: Passing 'suffixes' which cause duplicate columns {'idv_x', 'iso_id_x', 'comments_x', 'mas_x', 'pdi_x', 'valid_until_x', 'ivr_x', 'statenme_x', 'uai_x', 'ltowvs_x', 'cow_id_x', 'iso2_x', 'valid_from_x', 'country_x', 'cow3_x', 'cname_full_x', 'ctr_x', 'cname_x'} in the result is deprecated and will raise a MergeError in a future version.
  gravity_data = gravity_data.merge(culture_countries, left_on='importer', right_on='iso3').merge(culture_countries, left_on='exporter', right_on='iso3')


,importer,exporter,year,trade_value,agree_pta,common_language,contiguity,log_distance,IdealPointDistance,political_distance,ctr_x,country_x,pdi_x,idv_x,mas_x,uai_x,ltowvs_x,ivr_x,cow_id_x,cow3_x,iso_id_x,iso2_x,valid_from_x,valid_until_x,cname_x,cname_full_x,comments_x,statenme_x,ctr_y,country_y,pdi_y,idv_y,mas_y,uai_y,ltowvs_y,ivr_y,cow_id_y,cow3_y,iso_id_y,iso2_y,valid_from_y,valid_until_y,cname_y,cname_full_y,comments_y,statenme_y,ctr_x,country_x,pdi_x,idv_x,mas_x,uai_x,ltowvs_x,ivr_x,cow_id_x,cow3_x,iso_id_x,iso2_x,valid_from_x,valid_until_x,cname_x,cname_full_x,comments_x,statenme_x,ctr_y,country_y,pdi_y,idv_y,mas_y,uai_y,ltowvs_y,ivr_y,cow_id_y,cow3_y,iso_id_y,iso2_y,valid_from_y,valid_until_y,cname_y,cname_full_y,comments_y,statenme_y,ctr_x,country_x,pdi_x,idv_x,mas_x,uai_x,ltowvs_x,ivr_x,cow_id_x,cow3_x,iso_id_x,iso2_x,iso3_x,valid_from_x,valid_until_x,cname_x,cname_full_x,comments_x,statenme_x,ctr_y,country_y,pdi_y,idv_y,mas_y,uai_y,ltowvs_y,ivr_y,cow_id_y,cow3_y,iso_id_y,iso2_y,iso3_y,valid_from_y,valid_until_y,cname_y,cname_full_y,comments_y,statenme_y
0,AUS,BRA,1989,303546944.0,0.0,1.0,0.0,9.553332,1.299100,0.261671,AUL,Australia,38.0,90.0,61.0,51.0,21.0,71.0,900.0,AUL,36.0,AU,1962.0,NaN,Australia,Australia,NaN,Australia,BRA,Brazil,69.0,38.0,49.0,76.0,44.0,59.0,140.0,BRA,76.0,BR,1962.0,NaN,Brazil,Brazil,NaN,Brazil,AUL,Australia,38.0,90.0,61.0,51.0,21.0,71.0,900.0,AUL,36.0,AU,1962.0,NaN,Australia,Australia,NaN,Australia,BRA,Brazil,69.0,38.0,49.0,76.0,44.0,59.0,140.0,BRA,76.0,BR,1962.0,NaN,Brazil,Brazil,NaN,Brazil,AUL,Australia,38.0,90.0,61.0,51.0,21.0,71.0,900.0,AUL,36.0,AU,AUS,1962.0,NaN,Australia,Australia,NaN,Australia,BRA,Brazil,69.0,38.0,49.0,76.0,44.0,59.0,140.0,BRA,76.0,BR,BRA,1962.0,NaN,Brazil,Brazil,NaN,Brazil
1,AUS,BRA,1990,228831920.0,0.0,1.0,0.0,9.553332,1.519327,0.418267,AUL,Australia,38.0,90.0,61.0,51.0,21.0,71.0,900.0,AUL,36.0,AU,1962.0,NaN,Australia,Australia,NaN,Australia,BRA,Brazil,69.0,38.0,49.0,76.0,44.0,59.0,140.0,BRA,76.0,BR,1962.0,NaN,Brazil,Brazil,NaN,Brazil,AUL,Australia,38.0,90.0,61.0,51.0,21.0,71.0,900.0,AUL,36.0,AU,1962.0,NaN,Australia,Australia,NaN,Australia,BRA,Brazil,69.0,38.0,49.0,76.0,44.0,59.0,140.0,BRA,76.0,BR,1962.0,NaN,Brazil,Brazil,NaN,Brazil,AUL,Australia,38.0,90.0,61.0,51.0,21.0,71.0,900.0,AUL,36.0,AU,AUS,1962.0,NaN,Australia,Australia,NaN,Australia,BRA,Brazil,69.0,38.0,49.0,76.0,44.0,59.0,140.0,BRA,76.0,BR,BRA,1962.0,NaN,Brazil,Brazil,NaN,Brazil
2,AUS,BRA,1991,245040080.0,0.0,1.0,0.0,9.553332,1.631600,0.489561,AUL,Australia,38.0,90.0,61.0,51.0,21.0,71.0,900.0,AUL,36.0,AU,1962.0,NaN,Australia,Australia,NaN,Australia,BRA,Brazil,69.0,38.0,49.0,76.0,44.0,59.0,140.0,BRA,76.0,BR,1962.0,NaN,Brazil,Brazil,NaN,Brazil,AUL,Australia,38.0,90.0,61.0,51.0,21.0,71.0,900.0,AUL,36.0,AU,1962.0,NaN,Australia,Australia,NaN,Australia,BRA,Brazil,69.0,38.0,49.0,76.0,44.0,59.0,140.0,BRA,76.0,BR,1962.0,NaN,Brazil,Brazil,NaN,Brazil,AUL,Australia,38.0,90.0,61.0,51.0,21.0,71.0,900.0,AUL,36.0,AU,AUS,1962.0,NaN,Australia,Australia,NaN,Australia,BRA,Brazil,69.0,38.0,49.0,76.0,44.0,59.0,140.0,BRA,76.0,BR,BRA,1962.0,NaN,Brazil,Brazil,NaN,Brazil
3,AUS,BRA,1992,224085104.0,0.0,1.0,0.0,9.553332,1.563945,0.447212,AUL,Australia,38.0,90.0,61.0,51.0,21.0,71.0,900.0,AUL,36.0,AU,1962.0,NaN,Australia,Australia,NaN,Australia,BRA,Brazil,69.0,38.0,49.0,76.0,44.0,59.0,140.0,BRA,76.0,BR,1962.0,NaN,Brazil,Brazil,NaN,Brazil,AUL,Australia,38.0,90.0,61.0,51.0,21.0,71.0,900.0,AUL,36.0,AU,1962.0,NaN,Australia,Australia,NaN,Australia,BRA,Brazil,69.0,38.0,49.0,76.0,44.0,59.0,140.0,BRA,76.0,BR,1962.0,NaN,Brazil,Brazil,NaN,Brazil,AUL,Australia,38.0,90.0,61.0,51.0,21.0,71.0,900.0,AUL,36.0,AU,AUS,1962.0,NaN,Australia,Australia,NaN,Australia,BRA,Brazil,69.0,38.0,49.0,76.0,44.0,59.0,140.0,BRA,76.0,BR,BRA,1962.0,NaN,Brazil,Brazil,NaN,Brazil
4,AUS,BRA,1993,234342624.0,0.0,1.0,0.0,9.553332,1.048615,0.047470,AUL,Australia,38.0,90.0,61.0,51.0,21.0,71.0,900.0,AUL,36.0,AU,1962.0,NaN,Australia,Australia,NaN,Australia,BRA,Brazil,69.0,38.0,49.0,7

In [142]:
gravity_data['pdi_difference'] = np.abs(gravity_data['pdi_x'] - gravity_data['pdi_y'])
gravity_data['idv_difference'] = np.abs(gravity_data['idv_x'] - gravity_data['idv_y'])
gravity_data['mas_difference'] = np.abs(gravity_data['mas_x'] - gravity_data['mas_y'])
gravity_data['uai_difference'] = np.abs(gravity_data['uai_x'] - gravity_data['uai_y'])
gravity_data['ltowvs_difference'] = np.abs(gravity_data['ltowvs_x'] - gravity_data['ltowvs_y'])
gravity_data['ivr_difference'] = np.abs(gravity_data['ivr_x'] - gravity_data['ivr_y'])


ValueError: Expected a 1D array, got an array with shape (74856, 6)